In [2]:
"""
POC-2: Build a system to generate MOM from meeting transcripts
LLM: Ollama (llama3)
"""

from langchain_ollama import ChatOllama
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate


# -------------------------------------------------
# Load meeting transcript
# -------------------------------------------------
loader = TextLoader("meeting_transcript.txt")
documents = loader.load()

# -------------------------------------------------
# Chunk transcript
# -------------------------------------------------
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)
chunks = text_splitter.split_documents(documents)

# -------------------------------------------------
# Create embeddings
# -------------------------------------------------
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# -------------------------------------------------
# Store in FAISS
# -------------------------------------------------
vectorstore = FAISS.from_documents(
    chunks,
    embeddings
)

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 4}
)

# -------------------------------------------------
# LLM
# -------------------------------------------------
llm = ChatOllama(
    model="llama3",
    temperature=0
)

# -------------------------------------------------
# MOM Prompt
# -------------------------------------------------
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an assistant that generates clear and structured "
            "Minutes of Meeting (MOM) from meeting context."
        ),
        (
            "human",
            "Context:\n{context}\n\n"
            "Generate MOM with the following sections:\n"
            "- Meeting Date\n"
            "- Participants\n"
            "- Agenda\n"
            "- Key Decisions\n"
            "- Action Items"
        )
    ]
)

# -------------------------------------------------
# MOM Generator
# -------------------------------------------------
def generate_mom():
    docs = retriever.invoke("Generate minutes of meeting")
    context = "\n\n".join(doc.page_content for doc in docs)

    chain = prompt | llm
    response = chain.invoke({"context": context})
    return response.content


# -------------------------------------------------
# Run
# -------------------------------------------------
if __name__ == "__main__":
    mom = generate_mom()
    print("\n========== GENERATED MOM ==========\n")
    print(mom)

C:\Users\Shruti Mishra\AppData\Local\Temp\ipykernel_356\3637013886.py:32: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



========== GENERATED MOM ==========

Here is the Minutes of Meeting (MOM) based on the provided context:

**Meeting Date:** 12 July 2024

**Participants:**

* Product Manager
* Backend Engineer
* Frontend Engineer
* QA Lead

**Agenda:**

1. Discuss new checkout feature
2. Timeline planning
3. Identify blockers

**Key Decisions:**

* Redesign the checkout UI to improve user experience and optimize backend APIs for faster checkout flow.

**Action Items:**

1. Backend Engineer:
	* Optimize APIs by next week.
2. Frontend Engineer:
	* Create new UI mockups.
3. QA Lead:
	* Prepare test cases for the redesigned checkout UI.

Note: This MOM captures the key decisions and action items discussed during the meeting, providing a clear summary of the outcomes and responsibilities assigned to each participant.
